In [1]:
import itertools as it
import collections
import doctest
import string
import heapq
from typing import List, Tuple

import numpy as np

import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook"

In [2]:
import bs4
import urllib

def get_title(day):
    f = urllib.request.urlopen(f"https://adventofcode.com/2018/day/{day}")
    soup = bs4.BeautifulSoup(f.read())
    print(soup.h2.text.replace("-","").strip())    

## Day 1: Chronal Calibration

In [93]:
def day01_read(file_):
    return [int(line) for line in open(file_).readlines()]


def day01(xs: List[int]) -> int:
    return sum(xs)

In [94]:
inputs = day01_read("./2018/day01.txt")
day01(inputs)

497

In [95]:
def day01_mod(xs: List[int]) -> int:
    """
    >>> day01_mod([1, -1])
    0
    
    >>> day01_mod([+3, +3, +4, -2, -4])
    10
    
    >>> day01_mod([-6, +3, +8, +5, -6])
    5
    
    >>> day01_mod([+7, +7, -2, -7, -4])
    14
    """
    freq = 0
    seen = {freq}
    
    for x in it.cycle(xs):
        freq += x
        if freq in seen:
            break
        else:
            seen.add(freq)
    return freq

In [96]:
doctest.testmod()

TestResults(failed=0, attempted=8)

In [97]:
day01_mod(inputs)

558

## Day 2: Inventory Management System

### Part 1

Count number of appearences of a letter in a string.

In [102]:
def day02_read(file_: str) -> List[str]:
    return [line.strip() for line in open(file_).readlines()]


def day02(ss: List[str]) -> int:
    xs = [frozenset(collections.Counter(s).values()) for s in ss]
    twos = sum(1 for x in xs if 2 in x)
    threes = sum(1 for x in xs if 3 in x)
    return twos * threes

In [116]:
test = """
abcdef
bababc
abbcde
abcccd
aabcdd
abcdee
ababab""".split()

day02(test)

12

In [110]:
inputs = day02_read("./2018/day02.txt")
day02(inputs)

4940

In [111]:
len(inputs)

250

In [112]:
collections.Counter([len(s) for s in inputs])

Counter({26: 250})

So, there are 250 strings each of which has the same length 26. Now find a pair of the most similar strings. Here similarity is meant by number of common letters.

In [113]:
def day02_mod(ss: List[str]) -> str:

    def _similarity(pair):
        s, t = pair
        return sum(c1 == c2 for c1, c2 in zip(s, t))

    def _common_str(s, t):
        return "".join(c1 for c1, c2 in zip(s, t) if c1 == c2)

    size = len(ss)
    s, t = max([(s, t) for s, t in it.combinations(ss, 2)], 
              key=_similarity)
    return _common_str(s, t)

In [114]:
test = """
abcde
fghij
klmno
pqrst
fguij
axcye
wvxyz
""".split()

day02_mod(test)

'fgij'

In [115]:
day02_mod(inputs)

'wrziyfdmlumeqvaatbiosngkc'

## Day 3: No Matter How You Slice It

### Part 1

Place many rectangles of different height and width to specified grid locations. Then find the area where rectangles are overlapped.

In [118]:
def day03_read(file_):
    return [line.strip() for line in open("./2018/day03.txt").readlines()]
    
def day03_parse_line(line):
    """
    >>> day03_parse_line('#34 @ 23,699: 21x18')
    (34, 23, 699, 21, 18)
    """
    head, rest = line.split("@")
    id_ = int(head[1:])
    loc, size = rest.split(":")
    x, y = map(int, loc.split(","))
    dx, dy = map(int, size.split("x"))
    return id_, x, y, dx, dy

This is what an input looks like.

In [120]:
inputs = day03_read("./2018/day03.txt")
inputs[:4]

['#1 @ 871,327: 16x20',
 '#2 @ 676,717: 27x26',
 '#3 @ 245,818: 19x21',
 '#4 @ 89,306: 22x11']

In [129]:
## This one does not work for now. debugging...

def day03(lines):
    """
    >>> day03(["#1 @ 1,3: 4x4", "#2 @ 3,1: 4x4", "#3 @ 5,5: 2x2"])
    4
    """
    # いもす法
    d = collections.defaultdict(int)
    for line in lines:
        _, x, y, dx, dy = day03_parse_line(line)
        d[x, y] += 1
        d[x + dx, y     ] -= 1
        d[x     , y + dy] -= 1
        d[x + dy, y + dy] += 1
    
    ub_x = max(tup[0] for tup in d.keys()) + 1
    ub_y = max(tup[1] for tup in d.keys()) + 1
    
    res = 0
    for i in range(ub_x):
        for j in range(ub_y):
            d[i, j] += d[i, j - 1]

    xss = [[0] * (ub_x) for _ in range(ub_y)] 
    for j in range(ub_y):
        for i in range(ub_x):
            d[i, j] += d[i - 1, j]
            if d[i, j] >= 2:
                res += 1
            xss[j][i] = d[i, j]
            
#     for xs in xss:
#         print(xs)
    return res

In [130]:
def day03_naive(lines):
    """
    >>> day03_naive(["#1 @ 1,3: 4x4", "#2 @ 3,1: 4x4", "#3 @ 5,5: 2x2"])
    4
    """
    d = collections.defaultdict(int)
    for line in lines:
        _, x, y, dx, dy = day03_parse_line(line)
        for i in range(x, x + dx):
            for j in range(y, y + dy):
                d[i,j] += 1
    
    res = sum(1 for v in d.values() if v > 1)
    return res

In [121]:
doctest.testmod()

TestResults(failed=0, attempted=9)

In [122]:
day03_naive(inputs)

101565

### Part 2
Find ID of the rectangle that overlaps with none of the rest.

In [127]:
def day03_mod(lines):
    d = collaections.defaultdict(int)
    for line in lines:
        _, x, y, dx, dy = day03_parse_line(line)
        for i in range(x, x + dx):
            for j in range(y, y + dy):
                d[i, j] += 1

    for line in lines:
        id_, x, y, dx, dy = day03_parse_line(line)
        criteria = all(d[i, j] == 1 for i in range(x, x + dx) for j in range(y, y + dy))
        if criteria:
            return id_

In [128]:
day03_mod(inputs)

656

## Day 4: Repose Record

### Part 1

Keep track of the record of guard being awake or asleep at each minute. Find the guard that spent asleep most, and return the product of the guard ID and the minutes slept.

In [122]:
def day04_read(file_):
    return sorted([line.strip() for line in open(file_).readlines()])

In [123]:
inputs = day04_read("./2018/day04.txt")
inputs[:5]

['[1518-03-09 23:46] Guard #727 begins shift',
 '[1518-03-10 00:05] falls asleep',
 '[1518-03-10 00:34] wakes up',
 '[1518-03-10 00:56] falls asleep',
 '[1518-03-10 00:58] wakes up']

In [69]:
def _day04_parse(line):
    date = line[1:11]
    time = line[12:17]
    minute = int(time[-2:])
    msg = line[19:]
    return date, minute, msg

def _day04_guard_id(msg):
    return int(msg.split()[1][1:])

def day04(record):
    """    
    returns dictionary 
    {guard_id: 60-element list of number of minutes asleep at each time}
    
    いもす法
    """
    x = collections.defaultdict(lambda: [0]*60)
    guard = -1
    for line in record:
        date, minute, msg = _day04_parse(line)
        if msg.startswith("Guard"):
            guard = _day04_guard_id(msg)
        elif msg.startswith("falls"):
            x[guard][minute] += 1
        elif msg.startswith("wakes"):
            x[guard][minute] -= 1
        else:
            raise ValueError("Failed to decode message")

    for guard in x:
        x[guard] = list(it.accumulate(x[guard]))

    return x

In [70]:
x = day04(inputs)
guard = max(x.keys(), key=lambda g: sum(x[g]))
print(guard)
print(x[guard])
minute = max(range(60), key=lambda i: x[guard][i])
guard * minute

1987
[1, 1, 1, 2, 3, 3, 3, 3, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 7, 8, 9, 9, 10, 10, 10, 11, 11, 13, 13, 13, 13, 13, 13, 15, 13, 13, 13, 13, 13, 12, 11, 12, 12, 13, 12, 12, 11, 11, 11, 10, 11, 10, 7, 6, 4, 4, 4, 2, 0]


67558

### Part 2

Find the guard that was asleep at a clock time most. (There are many days with the same clock time.) Return the product of guard ID and the accumulated count of sleeps at the moment.

In [71]:
x = day04(inputs)
guard = max(x.keys(), key=lambda g: max(x[g]))
print(guard)
print(x[guard])
minute = max(range(60), key=lambda i: x[guard][i])
guard * minute

2633
[1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 6, 7, 6, 6, 6, 6, 6, 6, 6, 7, 7, 10, 12, 13, 14, 15, 17, 16, 15, 14, 12, 11, 11, 10, 10, 9, 7, 8, 8, 8, 9, 9, 7, 7, 7, 6, 6, 6, 7, 5, 5, 5, 3, 1, 1, 0]


78990

## Day 5: Alchemical Reduction

### Part 1

In [73]:
def day05_read(file_):
    return open(file_).read().strip()

In [74]:
inputs = day05_read("./2018/day05.txt")

An uppercase and a lowercase letter in adjacency react and dissapear if they are of the same kind (such as 'a' and 'A'). Chain reaction can happen. Find the number of letters after reactions.

In [75]:
def _day05_react(s):
    res = []
    c = None
    for c_next in s:
        if c is None:
            c = c_next
        elif c != c_next and c.lower() == c_next.lower():
            c = None
        else:
            res.append(c)
            c = c_next

    # collect the last element if necessary
    if c:
        res.append(c)
    return "".join(res)


def day05(s):
    """
    >>> day05("dabAcCaCBAcCcaDA")
    10
    """
    s1 = _day05_react(s)
    while s != s1:
        s = s1
        s1 = _day05_react(s)
    return len(s)

In [76]:
doctest.testmod()

TestResults(failed=0, attempted=7)

In [77]:
day05(inputs)

11546

### Part 2

Find the minimum length of letters after reactions when you remove one type of letters initially.

In [78]:
len(set(inputs.upper()))

26

In [79]:
def day05_mod(s):
    """
    >>> day05_mod("dabAcCaCBAcCcaDA")
    4
    """
    min_so_far = 1000000000
    for c in string.ascii_lowercase:
        s_mod = s.replace(c, "").replace(c.upper(), "")
        res = day05(s_mod)
        if res < min_so_far:
            min_so_far = res
    return min_so_far

In [80]:
doctest.testmod()

TestResults(failed=0, attempted=7)

In [81]:
day05_mod(inputs)

5124

## Day 6: Chronal Coordinates

### Part 1

I guess points NOT on the convex hull edges have finite areas.

In [125]:
import scipy.spatial

In [126]:
def day06_read(file_):
    return [tuple([int(x) for x in line.strip().split(",")])
            for line in open(file_).readlines()]

In [127]:
inputs = day06_read("./2018/day06.txt")
inputs[:10]

[(350, 353),
 (238, 298),
 (248, 152),
 (168, 189),
 (127, 155),
 (339, 202),
 (304, 104),
 (317, 144),
 (83, 106),
 (78, 106)]

In [133]:
def mins(iterable, key):
    iter1, iter2 = it.tee(iterable)
    minval = min(iter1, key=key)
    return [x for x in iter2 if key(x) == key(minval)]


def get_xy_range(points):
    xmin = 100000
    ymin = 100000
    xmax = -100000
    ymax = -100000
    for x, y in points:
        xmin = min(x, xmin)
        xmax = max(x, xmax)
        ymin = min(y, ymin)
        ymax = max(y, ymax)
    
    return xmin, xmax, ymin, ymax


def area_counts(points):
    d = dict()
    xmin, xmax, ymin, ymax = get_xy_range(points)
    for x in range(xmin, xmax+1):
        for y in range(ymin, ymax+1):
            target = x, y
            vals = mins(enumerate(points), key=lambda tup: _dist(tup[1], target))
            d[x,y] = vals[0][0] if len(vals) == 1 else None
    
    return collections.Counter(d.values())


def _dist(p1, p2):
    return sum(abs(i - j) for i, j in zip(p1, p2))


# def _explore(points, source, taken):
#     rest = [p for p in points if p != source]
#     seen = {source}
#     q = collections.deque([source])
#     count = 0
#     taken.add(source)

#     while q:
#         p = q.popleft()
#         d = _dist(source, p)
#         d_rest = min(_dist(another, p) for another in rest)
#         if d < d_rest:
#             count += 1
#             taken.add(p)
#         else:
#             continue
#         (x, y) = p
#         for next_p in [(x + 1, y), (x - 1, y), (x, y + 1), (x, y - 1)]:
#             if next_p not in seen:
#                 q.append(next_p)
#                 seen.add(next_p)

#     return count, taken


def day06(points):
    """
    >>> day06([(1, 1), (1, 6), (8, 3), (3, 4), (5, 5), (8, 9)])
    17
    """
    taken = set()
    hull = scipy.spatial.ConvexHull(points)
    indices_inner = [i for i in range(len(points)) if i not in set(hull.vertices)]
    counts = area_counts(points)
    print(counts, file=sys.stderr)
    return max(counts[i] for i in indices_inner)

In [134]:
doctest.testmod()

Counter({4: 17, 2: 12, 5: 10, 1: 9, 3: 9, None: 8, 0: 7})


TestResults(failed=0, attempted=10)

In [135]:
day06(inputs)

Counter({5: 3920, 6: 3807, 7: 3591, 24: 3374, 19: 3010, 46: 2957, 27: 2870, 22: 2516, 1: 2506, 28: 2468, 4: 2336, 34: 2325, 20: 2273, 15: 2266, 2: 2251, 10: 2180, 29: 2174, 39: 2097, 38: 2044, 12: 2025, 48: 2021, 16: 2019, 31: 1984, 42: 1836, 14: 1823, 47: 1729, 30: 1729, 23: 1700, 40: 1512, 44: 1471, 25: 1446, None: 1440, 3: 1387, 18: 1382, 41: 1228, 35: 1228, 43: 1214, 45: 1194, 36: 1192, 11: 1111, 33: 1044, 8: 1017, 49: 994, 13: 964, 26: 868, 17: 850, 37: 842, 9: 837, 21: 833, 0: 684, 32: 671})


3591

### Part 2

...

## Day 7: The Sum of Its Parts

### Part 1

Topological sorting of DAG

In [140]:
def day07_read(file_):
    return [line.strip() for line in open(file_).readlines()]

inputs = day07_read("./2018/day07.txt")
inputs[:3]

['Step O must be finished before step C can begin.',
 'Step Y must be finished before step D can begin.',
 'Step N must be finished before step D can begin.']

In [141]:
def _day07_parse_line(line):
    words = line.split()
    return words[1], words[-3]


def _day07_reorder(adj_list):
    res = dict()
    for k in sorted(adj_list.keys(), reverse=True):
        res[k] = sorted(adj_list[k], reverse=True)
    return res


def day07_get_adj_list(lines):
    adj_list = collections.defaultdict(list)
    for line in lines:
        from_, to_ = _day07_parse_line(line)
        adj_list[from_].append(to_)

    for x in set(it.chain.from_iterable(adj_list.values())):
        if x not in adj_list:
            adj_list[x] = []
        
    return adj_list

In [148]:
texts

['',
 'Step C must be finished before step A can begin.',
 'Step C must be finished before step F can begin.',
 'Step A must be finished before step B can begin.',
 'Step A must be finished before step D can begin.',
 'Step B must be finished before step E can begin.',
 'Step D must be finished before step E can begin.',
 'Step F must be finished before step E can begin.',
 '']

In [149]:
texts = [line for line in """
Step C must be finished before step A can begin.
Step C must be finished before step F can begin.
Step A must be finished before step B can begin.
Step A must be finished before step D can begin.
Step B must be finished before step E can begin.
Step D must be finished before step E can begin.
Step F must be finished before step E can begin.
""".split("\n") if line]

day07_get_adj_list(texts)

defaultdict(list,
            {'C': ['A', 'F'],
             'A': ['B', 'D'],
             'B': ['E'],
             'D': ['E'],
             'F': ['E'],
             'E': []})

In [145]:
inputs = day07_read("./2018/day07.txt")
adj_list = day07_get_adj_list(inputs)
adj_list

defaultdict(list,
            {'O': ['C', 'F', 'A', 'H', 'R', 'P'],
             'Y': ['D', 'Q', 'I'],
             'N': ['D', 'I', 'J', 'E', 'F'],
             'G': ['F', 'D', 'K', 'M', 'C', 'W', 'S'],
             'C': ['Z', 'D', 'U', 'X'],
             'H': ['K', 'S', 'D', 'P', 'Q'],
             'W': ['T', 'R', 'M', 'I'],
             'T': ['F', 'M', 'R', 'E', 'Q', 'L', 'V'],
             'S': ['I', 'Q', 'M', 'R', 'P', 'A', 'B'],
             'X': ['B', 'L', 'A'],
             'J': ['A', 'L', 'Q', 'F'],
             'K': ['D', 'Z', 'U', 'M'],
             'Z': ['A', 'B'],
             'A': ['B'],
             'L': ['V', 'B', 'R', 'E'],
             'F': ['M', 'D', 'Q', 'B', 'U'],
             'B': ['V', 'Q', 'P', 'R', 'I'],
             'M': ['Q', 'U', 'P', 'D'],
             'D': ['E', 'V', 'U', 'Q'],
             'I': ['U', 'Q', 'P', 'R'],
             'R': ['V', 'E', 'U', 'P'],
             'E': ['U', 'Q', 'V'],
             'P': ['V', 'U', 'Q'],
             'V': ['Q', 'U'],
  

In [150]:
def day07(adj_list):
    """
    >>> day07({"A": ["B", "D"], "B": [], "C": ["A", "F"], "D": ["E"], "E": [], "F": ["E"],})
    'CABDFE'
    """
    adj_list = _day07_reorder(adj_list)
    res = []
    seen = set()
    
    def _dps(x):
        for x_next in adj_list[x]:
            yield from _dps(x_next)
        yield x
    
    for k in adj_list.keys():
        if k in seen:
            continue
        else:
            for xx in _dps(k):
                if xx not in seen:
                    seen.add(xx)
                    res.append(xx)
    
    return "".join(reversed(res))

In [74]:
doctest.testmod()

TestResults(failed=0, attempted=5)

In [75]:
day07(adj_list)

'GNJOCHKSWTFMXLYDZABIREPVUQ'

### Part 2

Find the total time when DAG-constrained tasks are done by multiple workers under an optimal schedule. 

In [38]:
def get_parent(dag):
    parent = collections.defaultdict(list)
    for k, lis in dag.items():
        for v in lis:
            parent[v].append(k)
    return parent
    

def get_minimals(dag):
    minimals = set()
    parent = get_parent(dag)

    def _helper(x):
        if parent[x]:
            for x_p in parent[x]:
                yield from _helper(x_p)
        else:
            yield x

    res = set()
    for x in dag.keys():
        for y in _helper(x):
            res.add(y)
    return res

In [39]:
test = {"A": ["B", "D"], "B": [], "C": ["A", "F"], "D": ["E"], "E": [], "F": ["E"],}
get_minimals(test)

{'C'}

In [40]:
import heapq

In [92]:
def day07_mod(adj_list, n_workers):
    """
    >>> day07_mod({"A": ["B", "D"], "B": [], "C": ["A", "F"], "D": ["E"], "E": [], "F": ["E"],}, 2)
    375    
    """
    parent = get_parent(adj_list)
    print(parent)
    cost = {c: i for i, c in enumerate(string.ascii_uppercase, start=61)}
    workers = [(0, None)] * n_workers
    heapq.heapify(workers)

    h = []
    for c in get_minimals(adj_list):
        heapq.heappush(h, (-cost[c], c))
    
    done = set()
    t = 0
    while h:
        t, c = heapq.heappop(workers)
        print(f" ... a worker finished {c} at {t}")
        neg_dt, c = heapq.heappop(h)
        dt = -neg_dt
        print(f" ... next task {c} takes {dt}")
        heapq.heappush(workers, (t + dt, c))
        print(f" ... this worker will be available at {t + dt}")
        print("done:", done)
        
        for cc in adj_list[c]:
            task = (-cost[cc], cc)
            heapq.heappush(h, task) 
    return max(a for a, _ in workers)

In [93]:
doctest.testmod()

**********************************************************************
File "__main__", line 3, in __main__.day07_mod
Failed example:
    day07_mod({"A": ["B", "D"], "B": [], "C": ["A", "F"], "D": ["E"], "E": [], "F": ["E"],}, 2)
Expected:
    375    
Got:
    defaultdict(<class 'list'>, {'B': ['A'], 'D': ['A'], 'A': ['C'], 'F': ['C'], 'E': ['D', 'F']})
     ... a worker finished None at 0
     ... next task C takes 63
     ... this worker will be available at 63
    done: set()
     ... a worker finished None at 0
     ... next task F takes 66
     ... this worker will be available at 66
    done: set()
     ... a worker finished C at 63
     ... next task E takes 65
     ... this worker will be available at 128
    done: set()
     ... a worker finished F at 66
     ... next task A takes 61
     ... this worker will be available at 127
    done: set()
     ... a worker finished A at 127
     ... next task D takes 64
     ... this worker will be available at 191
    done: set()
     .

TestResults(failed=1, attempted=1)

In [3]:
get_title(8)

Day 8: Memory Maneuver


## Day 8: Memory Maneuver

### Part 1

Construct a tree from a sequence following rules. Return sum of node data.

In [139]:
def day08_read(file_):
    return [int(s) for s in open(file_).read().strip().split()]

inputs = day08_read("./2018/day08.txt")
inputs[:10]

[7, 11, 7, 3, 5, 5, 3, 5, 1, 8]

In [31]:
class Node(object):
    def __init__(self):
        self.data = []
        self.children = []

    def add_child(self, obj):
        self.children.append(obj)
    
    def add_data(self, d):
        self.data.append(d)

In [43]:
def _day08_get_node(iterable):
    n_children = next(iterable)
    n_data = next(iterable)
    node = Node()
    for _ in range(n_children):
        node.add_child(get_node(iterable))
    for _ in range(n_data):

        node.add_data(next(iterable))
    return node

In [65]:
def sum_tree(node):
    return sum(node.data) + sum(sum_tree(t) for t in node.children)

In [71]:
def day08(seq):
    """
    >>> test = [int(x) for x in "2 3 0 3 10 11 12 1 1 0 1 99 2 1 1 2".split()]
    >>> day08(test)
    138
    """
    n = _day08_get_node(iter(seq))
    return sum_tree(n)

In [72]:
day08(inputs)

35852

### Part 2

Find sum of "values" in a tree structure.

In [84]:
def _day08_get_value(node):
    if node.children:
        res = 0
        for i in node.data:
            i -= 1
            if 0 <= i < len(node.children):
                t = node.children[i]
                res += _day08_get_value(t)
    else:
        res = sum(node.data)
    return res


def day08_mod(seq):
    """
    >>> test = [int(x) for x in "2 3 0 3 10 11 12 1 1 0 1 99 2 1 1 2".split()]
    >>> day08_mod(test)
    66
    """
    n = _day08_get_node(iter(seq))
    return _day08_get_value(n)

In [85]:
doctest.testmod()

TestResults(failed=0, attempted=4)

In [86]:
day08_mod(inputs)

33422

In [87]:
get_title(9)

Day 9: Marble Mania


## Day 9: Marble Mania

### Part 1

Just implement rules of putting marbles in circle, and simulate.

In [274]:
def day09_read(file_: str) -> List[str]:
    return open(file_).read().strip()

inputs = day09_read("./2018/day09.txt")
inputs

'459 players; last marble is worth 71320 points'

In [257]:
class Ring(object):
    def __init__(self):
        self.state = [0]
        self.loc = 0

    def insert(self, x):
        assert x % 23 > 0
        self.loc = ((self.loc + 1) % (len(self.state)) + 1)
        self.state.insert(self.loc, x)
    
    def pop(self):
        self.loc = (self.loc - 7) % len(self.state)
        return self.state.pop(self.loc)
    
    def __repr__(self):
        return f"{self.state}       {self.loc}"            

In [258]:
r = Ring()
for i in range(1, 26):
    if i % 23 == 0:
        r_pop = r.pop()
        print(f"pop! {r_pop}")
    else:
        r.insert(i)
        print(r)

[0, 1]       1
[0, 2, 1]       1
[0, 2, 1, 3]       3
[0, 4, 2, 1, 3]       1
[0, 4, 2, 5, 1, 3]       3
[0, 4, 2, 5, 1, 6, 3]       5
[0, 4, 2, 5, 1, 6, 3, 7]       7
[0, 8, 4, 2, 5, 1, 6, 3, 7]       1
[0, 8, 4, 9, 2, 5, 1, 6, 3, 7]       3
[0, 8, 4, 9, 2, 10, 5, 1, 6, 3, 7]       5
[0, 8, 4, 9, 2, 10, 5, 11, 1, 6, 3, 7]       7
[0, 8, 4, 9, 2, 10, 5, 11, 1, 12, 6, 3, 7]       9
[0, 8, 4, 9, 2, 10, 5, 11, 1, 12, 6, 13, 3, 7]       11
[0, 8, 4, 9, 2, 10, 5, 11, 1, 12, 6, 13, 3, 14, 7]       13
[0, 8, 4, 9, 2, 10, 5, 11, 1, 12, 6, 13, 3, 14, 7, 15]       15
[0, 16, 8, 4, 9, 2, 10, 5, 11, 1, 12, 6, 13, 3, 14, 7, 15]       1
[0, 16, 8, 17, 4, 9, 2, 10, 5, 11, 1, 12, 6, 13, 3, 14, 7, 15]       3
[0, 16, 8, 17, 4, 18, 9, 2, 10, 5, 11, 1, 12, 6, 13, 3, 14, 7, 15]       5
[0, 16, 8, 17, 4, 18, 9, 19, 2, 10, 5, 11, 1, 12, 6, 13, 3, 14, 7, 15]       7
[0, 16, 8, 17, 4, 18, 9, 19, 2, 20, 10, 5, 11, 1, 12, 6, 13, 3, 14, 7, 15]       9
[0, 16, 8, 17, 4, 18, 9, 19, 2, 20, 10, 21, 5, 11, 1, 12, 6, 

In [279]:
def day09(num_players, last_marble):
    """
    >>> day09(9, 25)
    32
    >>> day09(10, 1618)
    8317
    >>> day09(13, 7999)
    146373
    >>> day09(17, 1104)
    2764
    >>> day09(21, 6111)
    54718
    >>> day09(30, 5807)
    37305
    """
    ring = Ring()
    player_points = [0] * num_players
    
    for marble in range(1, last_marble + 1):
        player = marble % num_players
        if marble % 23 == 0:
            r_pop = ring.pop()
            points = r_pop + marble
            player_points[player] += points
        else:
            ring.insert(marble)
    return max(player_points)

In [280]:
doctest.testmod()

Counter({4: 17, 2: 12, 5: 10, 1: 9, 3: 9, None: 8, 0: 7})


TestResults(failed=0, attempted=17)

In [282]:
day09(459, 71320)

375414

### Part 2

Find the same when number of marbles is 100 times larger.

In [284]:
get_title(10)

Day 10: The Stars Align


## Day 10: The Stars Align

### Part 1 & Part 2
Simulation of noninteracting particles in 2D.

In [3]:
def day10_read(file_: str) -> List[str]:
    return [line.strip() for line in open(file_).readlines()]

inputs = day10_read("./2018/day10.txt")
inputs[:3]

['position=<-50948,  20587> velocity=< 5, -2>',
 'position=< 20732, -51094> velocity=<-2,  5>',
 'position=<-30471, -10131> velocity=< 3,  1>']

In [4]:
def day10_parse_line(lines):
    pos = []
    vel = []
    for line in lines:
        pos_x = int(line[10:16])
        pos_y = int(line[18:24])
        vel_x = int(line[36:38])
        vel_y = int(line[40:42])
        pos.append((pos_x, pos_y))
        vel.append((vel_x, vel_y))
    return np.array(pos), np.array(vel)

In [5]:
x0, v = day10_parse_line(inputs)

In [18]:
# https://plot.ly/python/sliders/
import plotly.graph_objects as go

fig = go.Figure()

# Add traces, one for each slider step
for step in np.arange(10200, 10300, 2):
    pos = x0 + v * step 
    fig.add_trace(
        go.Scatter(
            visible=False,
#             line=dict(color="#00CED1", width=6),
            name=f"step = {step}",
            x=pos[:,0],
            y=pos[:,1],
            mode='markers',
        ))

# # Make 0th trace visible
fig.data[0].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="restyle",
        args=["visible", [False] * len(fig.data)],
    )
    step["args"][1][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "step: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.update_yaxes(autorange="reversed")


At t=10240, the points says "RLEZNRAN".

In [17]:
get_title(11)

Day 11: Chronal Charge


## Day 11: Chronal Charge

### Part 1

Sum of 3x3-square entries from 300x300 grid.

In [55]:
def day11_power_level(cell, serial_num):
    """
    >>> day11_power_level((3, 5), 8)
    4
    >>> day11_power_level((122, 79), 57)
    -5
    >>> day11_power_level((217, 196), 39)
    0
    >>> day11_power_level((101, 153), 71)
    4
    """
    x, y = cell
    rack_id = x + 10
    res = rack_id * y
    res += serial_num
    res *= rack_id
    res = int(str(res)[-3])
    res -= 5
    return res

def day11_total_power(cell, serial_num):
    """
    >>> day11_total_power((33, 45), 18)
    29
    >>> day11_total_power((21, 61), 42)
    30
    """
    x, y = cell
    res = sum(day11_power_level((x + i, y + j), serial_num)
            for i in range(3) for j in range(3))
    return res

In [56]:
doctest.testmod()

TestResults(failed=0, attempted=6)

In [52]:
res = [[0]*5 for _ in range(5)]
for i in range(5):
    for j in range(5):
        res[j][i] = day11_power_level((i + 32, j + 44), 18)

for line in res:
    print(" ".join(map(lambda n: f"{n: 2d}", line)))

-2 -4  4  4  4
-4  4  4  4 -5
 4  3  3  4 -4
 1  1  2  4 -3
-1  0  2 -5 -2


In [54]:
res = [[0]*5 for _ in range(5)]
for i in range(5):
    for j in range(5):
        res[j][i] = day11_power_level((i + 20, j + 60), 42)

for line in res:
    print(" ".join(map(lambda n: f"{n: 2d}", line)))

-3  4  2  2  2
-4  4  3  3  4
-5  3  3  4 -4
 4  3  3  4 -3
 3  3  3 -5 -1


In [64]:
import scipy.signal

In [97]:
def day11(serial_num):
    """
    >>> day11(18)
    (33, 45)
    >>> day11(42)
    (21, 61)
    """
    grid = np.zeros((300, 300))
    for i in range(300):
        for j in range(300):
            grid[i, j] = day11_power_level((i + 1, j + 1), serial_num)

    out = scipy.signal.convolve2d(grid, np.ones((3, 3)), mode='valid')
    idx = np.argmax(out)
    i, j = np.unravel_index(idx, out.shape)
    return (i + 1, j + 1)  # to 1-based indexing

In [95]:
doctest.testmod()

TestResults(failed=0, attempted=8)

In [96]:
day11(3613)

(20, 54)

### Part 2

Sum of NxN-square entries from 300x300 grid. 

**[FIXME]** This brute-force naive algorithm is slow.

In [103]:
def day11_mod(serial_num):
    """
    >>> day11(18)
    (90, 269, 16)
    >>> day11(42)
    (232, 251, 12)
    """
    grid = np.zeros((300, 300))
    for i in range(300):
        for j in range(300):
            grid[i, j] = day11_power_level((i + 1, j + 1), serial_num)

    d = dict()
    for size in range(1, 301):
        out = scipy.signal.convolve2d(grid, np.ones((size, size)), mode='valid')
        idx = np.argmax(out)
        i, j = np.unravel_index(idx, out.shape)
        d[i, j, size] = int(out[i, j])
    
    i, j, size = max(d.keys(), key=lambda k: d[k])
    return (i + 1, j + 1, size)  # to 1-based indexing


In [105]:
day11_mod(3613)

(233, 93, 13)

In [106]:
get_title(12)

Day 12: Subterranean Sustainability


## Day 12: Subterranean Sustainability

### Part 1

In [107]:
def day12_read():


In [108]:
inputs = day12_read()

FileNotFoundError: [Errno 2] No such file or directory: './2019/day12.txt'